In [23]:
import requests
def download_data()->dict: #傳出dict
    url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON'

    response = requests.get(url) #response是實體
    if response.status_code == 200:
        print('下載成功')
    return response.json()

In [24]:
def jsonDict_csvList(json)->list[dict]:
    '''
    - 由online json viewer看出locaion下的21個地點資料都是dict
    - 傳入json的資料結構，並取出需要的資料
    - 把json資料結構轉成python list，而內部是dict結構
    '''
    location = json['cwbopendata']['dataset']['location'] #location是一個list資料結構，是我們要的，因此用變數location來接收
    weather_list = [] #希望建立外部為list，而內部是dict的資料結構
    for item in location: #既然location是list也就是sequence，因此可以用for in來抓資料
        city_item = {} #希望建立內部是dict的資料結構
        city_item['城市'] = item['locationName']
        city_item['起始時間'] = item['weatherElement'][1]['time'][0]['startTime'] #但由online json viewer看出weatherElement是一個索引編號，是一個list結構
        city_item['結束時間'] = item['weatherElement'][1]['time'][0]['endTime']
        city_item['最高溫度'] = float(item['weatherElement'][1]['time'][0]['parameter']['parameterName'])
        city_item['最低溫度'] = float(item['weatherElement'][2]['time'][0]['parameter']['parameterName'])
        city_item['感覺'] = item['weatherElement'][3]['time'][0]['parameter']['parameterName']
        weather_list.append(city_item)
    return weather_list


In [32]:
import csv
def save_csv(data:list[dict],fileName)->bool:
    '''
    - 將list[dict]儲存
    - 參數fileName要儲存的檔案名
    '''
    with open(fileName,mode='w',encoding='utf-8',newline='') as file:  #csv檔案需要多加一個newline=''
        fieldnames = ['城市','起始時間','結束時間','最高溫度','最低溫度','感覺']
        writer = csv.DictWriter(file,fieldnames=fieldnames)
        writer.writeheader() #header就是'城市','起始時間','結束時間','最高溫度','最低溫度','感覺'
        writer.writerows(data)
        return True

In [26]:
json_data = download_data() #下載資料
csv_list = jsonDict_csvList(json_data) #解析json的資料成為csv的結構
save_csv(csv_list,"今天天氣.csv") #儲存為csv資料

下載成功


True

In [27]:
from datetime import datetime
import pytz
def get_csvName()->str:
    '''
    - 取得台灣目前year-month-day.csv
    '''
    taiwan_timezone = pytz.timezone('Asia/Taipei')
    current_date = datetime.now(taiwan_timezone)
    fileName = f"{current_date.year}-{current_date.month}-{current_date.day}.csv"
    return fileName

In [34]:
import os
def get_fileName_path()->str:
    csv_file_name = get_csvName()
    current_cwd = os.path.abspath(os.getcwd())
    abs_file_path = os.path.join(current_cwd,'data',csv_file_name)
    return abs_file_path

def check_file_exist()->bool:
    abs_file_path = get_fileName_path()
    if os.path.exists(abs_file_path):
        return True
    else:
        return False

check_file_exist()

False

In [37]:
if not check_file_exist():
    print('檔案不存在')
    json_data = download_data()
    csv_list = jsonDict_csvList(json_data)
    is_save = save_csv(csv_list,get_fileName_path())
    if is_save:
        print('存檔成功')


檔案不存在
下載成功
/workspaces/python-chihlee-class/taiwan_weather/data/2023-8-19.csv
存檔成功
